In [1]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset

c:\Users\TVT\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load pre-trained T5 model and tokenizer
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
# Load dataset for fine-tuning (e.g., CNN/DailyMail dataset)
dataset = load_dataset("cnn_dailymail", "3.0.0")
small_dataset = dataset["train"].select(range(1000))  # Select the first 100 examples

In [12]:
# Tokenize dataset for training
def tokenize_function(example):
    source_text = example["article"]
    target_text = example["highlights"]
    source_tokenized = tokenizer(source_text, truncation=True, padding="max_length", max_length=1024, return_tensors="pt")
    target_tokenized = tokenizer(target_text, truncation=True, padding="max_length", max_length=150, return_tensors="pt")
    return {
        "input_ids": source_tokenized.input_ids,
        "attention_mask": source_tokenized.attention_mask,
        "labels": target_tokenized.input_ids,
    }

tokenized_datasets = small_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 1000/1000 [00:02<00:00, 431.78 examples/s]


In [13]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    logging_dir="./logs",
    logging_steps=1000,
)

# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
)

c:\Users\TVT\AppData\Local\Programs\Python\Python311\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [14]:
# Fine-tune the model
trainer.train()

100%|██████████| 750/750 [2:31:32<00:00, 12.12s/it]  

{'train_runtime': 9092.9189, 'train_samples_per_second': 0.33, 'train_steps_per_second': 0.082, 'train_loss': 0.9247189127604166, 'epoch': 3.0}


TrainOutput(global_step=750, training_loss=0.9247189127604166, metrics={'train_runtime': 9092.9189, 'train_samples_per_second': 0.33, 'train_steps_per_second': 0.082, 'train_loss': 0.9247189127604166, 'epoch': 3.0})

In [ ]:
# Save the fine-tuned model
model.save_pretrained("./fine_tuned_t5_small")

In [15]:
# Example of generating summaries using the fine-tuned model
input_text = """Artificial intelligence (AI) is a field of computer science that aims to create systems capable of performing tasks that typically require human intelligence. 
The concept of AI dates back to ancient times, with early ideas emerging in Greek mythology and ancient Greek philosophy. However, the modern era of AI began in the mid-20th 
century with the development of computer technology and the advent of digital computing. In 1956, the term "artificial intelligence" was coined at the Dartmouth Conference,
 where researchers gathered to discuss the potential of creating machines that could mimic human cognitive abilities. Since then, AI has evolved rapidly, with significant 
 advancements in areas such as machine learning, natural language processing, computer vision, and robotics. AI technologies have been applied across various industries, 
 including healthcare, finance, transportation, and entertainment, revolutionizing the way we live and work. From virtual assistants like Siri and Alexa to self-driving cars
   and advanced medical diagnostic systems, AI has become an integral part of our daily lives. However, AI also raises ethical and societal concerns, including issues related
     to privacy, bias, job displacement, and the potential for misuse of AI-powered systems. Despite these challenges, the pursuit of artificial intelligence continues to drive
       innovation and shape the future of technology and society.
"""
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
generated_summary_ids = model.generate(input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
generated_summary = tokenizer.decode(generated_summary_ids[0], skip_special_tokens=True)
print("Generated Summary:")
print(generated_summary)

Generated Summary:
AI is a field of computer science that aims to create systems capable of performing tasks. The concept of AI dates back to ancient times, with early ideas emerging in Greek mythology and ancient Greek philosophy. AI has been applied across various industries, including healthcare, finance, transportation, and entertainment.
